In [1]:
from typing import Tuple

import numpy as np

import minitorch.autodiff.tensor_functions as tf
from minitorch import operators
from minitorch.module import LinearTensorLayer
from minitorch.autodiff import Context, Tensor, topological_sort
from minitorch.autodiff.tensor_ops import SimpleBackend

In [2]:
input_dim, output_dim = 2, 1
linear = LinearTensorLayer(input_dim, output_dim)
weights, bias = linear._weights.value, linear._bias.value

# Generate some input data
n_samples = 10
inputs = tf.rand((n_samples, input_dim))
targets = tf.tensor([1, 1, 1, 0, 0, 0, 1, 0, 1, 0])

# Forward
inputs = inputs.view(*inputs.shape, 1)
_weights = weights.view(1, *weights.shape)

out = (inputs * _weights).sum(dim=1)
predictions = out.view(inputs.shape[0], bias.size) + bias
predictions = predictions.sigmoid()

# Compute loss
probas = (predictions * targets) + (predictions - 1.0) * (targets - 1.0)
loss = ((-probas.log()) / targets.size).sum()
loss.backward()


In [7]:
weights.derivative


[
	[-1.40137]
	[-0.96990]
]

### Compare to torch implementation

In [5]:
# Compare to torch implementation

(2, 1)

(1,)

In [ ]:
we